In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
import boto3
import os
import tarfile
import numpy as np
from PIL import Image
import json
from torchvision import transforms


In [ ]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [ ]:
# Create tar.gz file w/ the model and inference script
def create_model_tarball():
    with tarfile.open('model.tar.gz', 'w:gz') as tar:
        tar.add('best_model.pth')
        tar.add('code/inference.py')

create_model_tarball()

In [ ]:
# Upload the model.tar.gz to S3
model_data = sagemaker_session.upload_data(
    'model.tar.gz',
    key_prefix='pathmnist-model'
)

In [ ]:
# Create PyTorch model
pytorch_model = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point='code/inference.py',
    framework_version='2.3.0',
    py_version='py311',
    predictor_cls=sagemaker.pytorch.model.PyTorchPredictor
)

In [ ]:
# Deploy model
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name='pathmnist-endpoint',
    client_config={'request_timeout': 120}
)

In [ ]:
sagemaker_client = boto3.client('sagemaker')

response = sagemaker_client.describe_endpoint(EndpointName='pathmnist-endpoint')
print("Endpoint Status:", response['EndpointStatus'])

In [ ]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
    ])
    
    image = Image.open(image_path)
    image_tensor = transform(image)
    
    # Convert to numpy array, shape (3, 28, 28)
    return image_tensor.numpy()

def predict_single_image(image_path, endpoint_name):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    
    image_tensor = transform(image)
    image_array = image_tensor.numpy()
    
    print(f"Image shape: {image_array.shape}") # expect (3, 28, 28)
    print(f"Value range: [{image_array.min():.3f}, {image_array.max():.3f}]")
    
    runtime = boto3.client('sagemaker-runtime')
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=json.dumps(image_array.tolist())
    )
    
    prediction = json.loads(response['Body'].read().decode())
    return prediction



In [ ]:
# Test it out
prediction = predict_single_image('sample_0_label_[8].png', 'pathmnist-endpoint')
prediction

In [ ]:
# delete the endpoint when done
predictor.delete_endpoint()